# Preparing Basis States in PennyLane

## Importing PennyLane and the wrapped version of NumPy

PennyLane is an open source software for quantum machine learning and quantum computing algorithms. It integrates standard machine learning software and tools, such as TensorFlow and Pytorch, with quantum computing software like IBM's Qiskit, Microsoft's Q#, and Google's Cirq. It also has its own built in functions and models, and allows various hardware backends such as the quantum computers available from IBM via free cloud access. PennyLane has a wrapped version of NumPy that we import rather than the standard NumPy we have been using. 

In [1]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^


In [2]:
import pennylane as qml
from pennylane import numpy as np

## Basis States

In PennyLane, and other quantum computing software, we often want to prepare basis states. This is something we will return to again in the future when we study specific cases of quantum circuits, but having some basic introduction to state preparation now seems appropriate since the are simply tensor products of the basis states $|0\rangle$ and $|1\rangle$. In particular, we can prepare states like

\begin{align}
|00 \rangle &= |0\rangle \otimes |0\rangle = \begin{pmatrix} 1\\0 \end{pmatrix} \otimes \begin{pmatrix} 1\\0 \end{pmatrix} = \begin{pmatrix} 1\\0\\0\\0 \end{pmatrix} \\
|01 \rangle &= |0\rangle \otimes |1\rangle = \begin{pmatrix} 1\\0 \end{pmatrix} \otimes \begin{pmatrix} 0\\1 \end{pmatrix} = \begin{pmatrix} 0\\1\\0\\0 \end{pmatrix} \\
|10 \rangle &= |1\rangle \otimes |0\rangle = \begin{pmatrix} 0\\1 \end{pmatrix} \otimes \begin{pmatrix} 1\\0 \end{pmatrix} = \begin{pmatrix} 0\\0\\1\\0 \end{pmatrix} \\
|11 \rangle &= |1\rangle \otimes |1\rangle = \begin{pmatrix} 0\\1 \end{pmatrix} \otimes \begin{pmatrix} 0\\1 \end{pmatrix} = \begin{pmatrix} 0\\0\\0\\1 \end{pmatrix}
\end{align}

Let's define these in Python

In [4]:
# Define single qubit states spin-up and spin-down
u = np.matrix([[1],
               [0]])
d = np.matrix([[0],
               [1]])

# Shows u 
print(u)

# Shows d
print(d)

# Define the basis states:
uu = np.kron(u, u)
ud = np.kron(u, d)
du = np.kron(d, u)
dd = np.kron(d, d)


[[1]
 [0]]
[[0]
 [1]]


In [6]:
# Shows uu
print('|00> =')
print(uu)

|00> =
[[1]
 [0]
 [0]
 [0]]


In [7]:
# Shows ud
print('|01> =')
print(ud)

|01> =
[[0]
 [1]
 [0]
 [0]]


In [8]:
# Shows du
print('|10> =')
print(du)

|10> =
[[0]
 [0]
 [1]
 [0]]


In [9]:
# Shows dd
print('|11> =')
print(dd)

|11> =
[[0]
 [0]
 [0]
 [1]]


Taking this further, we can define states like:
    
\begin{align}
|000\rangle = |0\rangle \otimes |0\rangle \otimes |0\rangle = \begin{pmatrix} 1\\0\\0\\0\\0\\0\\0\\0 \end{pmatrix}
\end{align}

\begin{align}
|010\rangle = |0\rangle \otimes |1\rangle \otimes |0\rangle = \begin{pmatrix} 0\\0\\1\\0\\0\\0\\0\\0 \end{pmatrix}
\end{align}

In PennyLane the following code defines a device (quantum computer or simulator) on which to run code. We will use the default simulator for now since the code is very simple and running it on an actual quantum computer is unnecessary. The number of wires is equal to the number of qubits we are using. The number of shots is the number of times we want to run the circuit.

In [10]:
# Creating a default qubit simulator
dev = qml.device("default.qubit", wires=2, shots=1)

Now we can define an array that corresponds to the basis state $|00\rangle$:

In [11]:
# 00 basis state 
uu = np.array([0, 0])

We can define a function `circuit()` that defines a quantum circuit. 

In [12]:
# Defining a quantum circuit 
def circuit():
    qml.BasisState(uu, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

Now, we define a `qnode` class that runs on the device we have defined and samples the qubits and gives a $+1$ for spin-up and a $-1$ for spin down. This runs a quantum function (circuit) that we have just defined. 

In [13]:
# Creating a qnode class
@qml.qnode(dev)
def circuit():
    qml.BasisState(uu, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

(array(1, dtype=int64), array(1, dtype=int64))


Since qubits are by default in the spin-up state for any quantum circuit, and since we did not prepare any special initial state or perform any operations on the qubits, we get an expectation value of $+1$ for both when sampling the circuit. The vector above show a $+1$ in the first and second entry meaning it is measuring two spin-up states. Let's define a new state to prepare corresponding to $|01 \rangle$:

In [14]:
# 01 basis state 
ud = np.array([0,1])

Next, let's define a new `qnode` to sample corresponding to this prepared state:

In [15]:
# A qnode class
@qml.qnode(dev)
def circuit():
    qml.BasisState(ud, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

(array(1, dtype=int64), array(-1, dtype=int64))


As we can see, we get a $+1$ in the first component, corresponding to the spin-up state, and we get a $-1$ in the second component corresponding to the spin-down state. We can also tell PennyLane to perform multiple "shots" or samples by defining a new device:

In [17]:
# Creating a new default qubit simulator
dev2 = qml.device("default.qubit", wires=2, shots=10)

In [18]:
# A new qnode class
@qml.qnode(dev2)
def circuit():
    qml.BasisState(ud, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int64))


This gives us 10 output samples instead of just one. If we define a new array corresponding to the state $|11\rangle$,

In [16]:
# 11 basis state 
dd = np.array([1,1])

We can define a new 'qnode' with the new device we just defined, and we should expect an output vector of 

\begin{align}
\begin{pmatrix}
-1\\-1
\end{pmatrix}
\end{align}

ten times, corresponding to the ten samples:

In [17]:
@qml.qnode(dev2)
def circuit():
    qml.BasisState(dd, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


We will return to this when we discuss measurements and expectation values. 

## Exercises

### Write code to compute the following basis states:

1. $|000 \rangle$
2. $|001 \rangle$
3. $|010 \rangle$
4. $|011 \rangle$
5. $|100 \rangle$
6. $|101 \rangle$
7. $|110 \rangle$
8. $|111 \rangle$

Now, compute these by hand to verify your code is correct. For example, the first computation will be

\begin{align}
\begin{pmatrix} 1\\0 \end{pmatrix} \otimes
\begin{pmatrix} 1\\0 \end{pmatrix} \otimes
\begin{pmatrix} 0\\1 \end{pmatrix}
\end{align}

9. Define a defaul device with "3 wires" and "5 shots".
10. Define an array 'uud' for the basis state $|001\rangle$.
11. Now define a `qnode` that will print the samples: 

> `qml.sample(qml.PauliZ(0))`

> `qml.sample(qml.PauliZ(1))` 

> `qml.sample(qml.PauliZ(2))`

12. Run your 'qnode' and verify that you get the output vector
\begin{align}
\begin{pmatrix}
1 \\ 1 \\ -1
\end{pmatrix}
\end{align}

five times.

13. Try doing this with the other basis states given in Exercises 1-8.

In [25]:
# 1

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |000> state
@qml.qnode(dev)
def circuit_000():
    uu = np.array([0, 0, 0])  # |000> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |000>
state_000 = circuit_000()
print("State |000> (tensor product of qubits):", state_000)


State |000> (tensor product of qubits): [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [26]:
# 2

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |001> state
@qml.qnode(dev)
def circuit_001():
    uu = np.array([0, 0, 1])  # |001> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |001>
state_001 = circuit_001()
print("State |001> (tensor product of qubits):", state_001)

State |001> (tensor product of qubits): [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [27]:
# 3

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |010> state
@qml.qnode(dev)
def circuit_010():
    uu = np.array([0, 1, 0])  # |010> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |010>
state_010 = circuit_010()
print("State |010> (tensor product of qubits):", state_010)

State |010> (tensor product of qubits): [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [28]:
# 4

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |011> state
@qml.qnode(dev)
def circuit_011():
    uu = np.array([0, 1, 1])  # |011> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |011>
state_011 = circuit_011()
print("State |011> (tensor product of qubits):", state_011)

State |011> (tensor product of qubits): [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [29]:
# 5

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |100> state
@qml.qnode(dev)
def circuit_100():
    uu = np.array([1, 0, 0])  # |100> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |100>
state_100 = circuit_100()
print("State |100> (tensor product of qubits):", state_100)

State |100> (tensor product of qubits): [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [30]:
# 6

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |101> state
@qml.qnode(dev)
def circuit_101():
    uu = np.array([1, 0, 0])  # |100> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |101>
state_101 = circuit_101()
print("State |101> (tensor product of qubits):", state_101)

State |101> (tensor product of qubits): [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [31]:
# 7

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |110> state
@qml.qnode(dev)
def circuit_110():
    uu = np.array([1, 0, 0])  # |110> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |101>
state_110 = circuit_110()
print("State |110> (tensor product of qubits):", state_110)

State |110> (tensor product of qubits): [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [32]:
# 8

# Create a default qubit simulator with 3 qubits
dev = qml.device("default.qubit", wires=3, shots=None)

# Defining a quantum circuit for |111> state
@qml.qnode(dev)
def circuit_111():
    uu = np.array([1, 0, 0])  # |111> basis state
    qml.BasisState(uu, wires=[0, 1, 2])
    return qml.state()  # Return the full quantum state (the tensor product of all qubits)

# Run and print the result for |111>
state_111 = circuit_111()
print("State |111> (tensor product of qubits):", state_111)

State |111> (tensor product of qubits): [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [33]:
# 9 - 12

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |001> (which is 'uud' in your description)
uud = np.array([0, 0, 1])  # Corresponds to |001>

# Define a quantum circuit that sets the qubits in the |001> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |001> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)


Samples from the Pauli-Z measurements on each qubit:
 [array([1, 1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64)]


In [36]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |000> (which is 'uud' in your description)
uud = np.array([0, 0, 0])  # Corresponds to |000>

# Define a quantum circuit that sets the qubits in the |000> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |000> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([1, 1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64)]


In [37]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |110> (which is 'uud' in your description)
uud = np.array([1, 1, 0])  # Corresponds to |110>

# Define a quantum circuit that sets the qubits in the |110> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |110> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([-1, -1, -1, -1, -1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64)]


In [38]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |111> (which is 'uud' in your description)
uud = np.array([1, 1, 1])  # Corresponds to |111>

# Define a quantum circuit that sets the qubits in the |111> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |111> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([-1, -1, -1, -1, -1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64)]


In [39]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |010> (which is 'uud' in your description)
uud = np.array([0, 1, 0])  # Corresponds to |010>

# Define a quantum circuit that sets the qubits in the |010> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |010> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([1, 1, 1, 1, 1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64)]


In [50]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |011> (which is 'uud' in your description)
uud = np.array([0, 1, 1])  # Corresponds to |011>

# Define a quantum circuit that sets the qubits in the |011> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |011> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([1, 1, 1, 1, 1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64)]


In [43]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |100> (which is 'uud' in your description)
uud = np.array([1, 0, 0])  # Corresponds to |100>

# Define a quantum circuit that sets the qubits in the |100> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |100> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([-1, -1, -1, -1, -1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64)]


In [49]:
# 13

# Create a default qubit simulator with 3 wires and 5 shots
dev = qml.device("default.qubit", wires=3, shots=5)

# Define the basis state |101> (which is 'uud' in your description)
uud = np.array([1, 0, 1])  # Corresponds to |101>

# Define a quantum circuit that sets the qubits in the |101> state and samples Pauli-Z measurements
@qml.qnode(dev)
def circuit():
    qml.BasisState(uud, wires=[0, 1, 2])  # Prepare the |101> state
    return [
        qml.sample(qml.PauliZ(0)),  # Measure qubit 0
        qml.sample(qml.PauliZ(1)),  # Measure qubit 1
        qml.sample(qml.PauliZ(2))   # Measure qubit 2
    ]

# Run the quantum circuit and print the result
samples = circuit()
print("Samples from the Pauli-Z measurements on each qubit:\n", samples)

Samples from the Pauli-Z measurements on each qubit:
 [array([-1, -1, -1, -1, -1], dtype=int64), array([1, 1, 1, 1, 1], dtype=int64), array([-1, -1, -1, -1, -1], dtype=int64)]
